In [68]:
import numpy
import datetime
import re
import collections
from collections import OrderedDict

In [28]:
##
## STRING DEFINITIONS - constant strings of files that were used and additional info that couldn't be pulled from the files
##

reference_file_str = '/sbgenomics/project-files/human_g1k_v37_decoy.fasta'
reference_fai_file_str = '/sbgenomics/project-files/human_g1k_v37_decoy.fasta.fai'
#pileup_file_str = '/sbgenomics/project-files/example_human_pileup' #rename it to merged_normal-example_human_ref-pileup
pileup_file_str = 'algo_test_pileup' #rename it to merged_normal-example_human_ref-pileup
output_file_temp_str = 'temp.vcf'
output_file_str = 'binom_call-merged_normal.vcf'

ref_assembly_str = ',assembly=b37'
ref_md5_str = ',md5=bb77e60e9a492fd0172e2b11e6c16afd'
ref_species_str = ',species=\"Homo sapiens\"'
ref_taxonomy_str = ',taxonomy=x' #couldn't find what this acctually means


In [29]:
##
## VCF CONTIG INFO FUNC - Pulls out information on all contigs from reference .fasta.fai file
##

def write_contig_info(file):
    contig_info =''
    for line in file:
        field = line.split("\t")
        ID_str = field[0]
        len_str = field[1]
        contig_info += '##contig=<ID=' + ID_str + ",length=" + len_str + ref_assembly_str + ref_md5_str + ref_species_str + ref_taxonomy_str + '>' + '\n'
    return contig_info
        
        

In [30]:
##
## VCF HEADER FUNCTION - Writes out header of the .vcf file
##

def write_vcf_header(file, fai):
    date_time = datetime.datetime.now()
    
    file_format_str = '##fileformat=VCF4.2\n'
    date_str = '##fileDate=' + date_time.strftime("%Y%m%d") + '\n'
    source_str = '##source=variant_call_binomV0.1\n'
    ref_file_str = '##reference=file://' + reference_file_str + '\n'
    contig_info_str = write_contig_info(fai)    
    #filter_string = '##FILTER=<ID=PASS,Description=\"\"\n' #TODO check if this is necessary
    alt_str = '##ALT=<ID=*,Description="Represents allele(s) other than observed.">\n'
    table_header_str = '#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tHCC1143BL\n'
    
    file.write(file_format_str + date_str + source_str + ref_file_str + contig_info_str + alt_str + table_header_str)
    
    fai.close()
    return file


In [117]:
##
## CALCULATE VARIANTS CALLS
##

class Variant:
    def __init__(self, symbol, num_occ, num_reads):
        self.var_type, self.var_symbol, self.var_len = self.variant_def(symbol)
        
        self.num_occ = num_occ
        self.occ_percent = float(num_occ/num_reads)
        print("var")
        
    
    def variant_def(self, symbol):
        if ( len(symbol) == 1):
                var_symbol = symbol                
                if(symbol == '.'):
                    var_type = 'REF'
                else:
                    var_type = 'SNV'
                var_len = 1
        else:
            if(symbol[1]== '+'):
                var_type = 'INS'
            else:
                var_type = 'DEL'
            var_symbol = ''.join(filter(str.isalpha, symbol))
            var_len = int(''.join(filter(str.isdigit, symbol)))
    
        return var_type, var_symbol, var_len
    
        


In [205]:
##
## STORE SINGLE PILEUP CALLS
##

#TODO 
        # OPTIONAL: calculate all the qualities of the reads
        # remove all carets and dollars
        # if there are insertions or deletions, create new reads for them
        # count all different reads (OPTIONAL: and add specific base qualities)
        # based on the reads decide on two best variants
        # pass variants back

class PileupLine:
    def __init__(self, new_line):
        field = new_line.split("\t")
        self.seq_str = field[0] # sequence name string
        self.pos_str = field[1] # position string
        self.ref_str = field[2] # reference base string
        self.cnt_str = field[3] # read count string
        self.res_str = field[4] # result string
        self.qual_str = field[5] # base quality string
        
        self.pos = int(self.pos_str)
        self.cnt = int(self.cnt_str)
        
        self.dot_cnt = 0
        self.a_cnt = 0
        self.t_cnt = 0
        self.c_cnt = 0
        self.g_cnt = 0
        self.ins_cnt = 0
        self.del_cnt = 0
        
        self.ins_seqs  = {}
        self.del_seqs = {}
     
        self.read_vars = collections.OrderedDict()
        self.read_vars = {
            '.':0,
            'A':0,
            'T':0,
            'C':0,
            'G':0
        }
        
    
    def find_variants (self, read_str):
        
        var1 = None
        var2 = None
        
        has_ins = read_str.find('+')
        has_del = read_str.find('-')
        
        read_str = self.prepare_read_str(read_str)
#         read_str = re.sub('\^.','',read_str)
#         read_str = re.sub('\$|\*','',read_str)
            
        if ((has_ins == -1) and (has_del == -1)):            
            sym_cnt = collections.Counter(read_str.strip('"')).most_common()
            
            print('find_variants')

            #if (len(sym_cnt) == 0): should not happen    
            if (len(sym_cnt) == 1):
                var1 = Variant(sym_cnt[0][0], sym_cnt[0][1], self.cnt)
                var2 = None
                print(var1.var_type)
            else:
                var1 = Variant(sym_cnt[0][0], sym_cnt[0][1], self.cnt)
                var2 = Variant(sym_cnt[1][0], sym_cnt[1][1], self.cnt)
                print(var1.var_type)
                print(var2.var_type)
        
        elif (has_ins != -1 or has_del != -1):
            self.prepare_read_vars(read_str)
            if(self.read_vars_list.[0][1] == 0):
                print('No vars')
            elif(self.read_vars_list[1][1] == 0):
                print('Single var')
                print(self.read_vars_list[1][0])
            else(self.read_vars_list.[1][1] == 0):
                print('Two vars')
                print(self.read_vars_list[0][0])
                print(self.read_vars_list[1][0])
        
        return var1, var2
        
           
    def prepare_read_str(self, read_str): #raname to calculate_variants
        
        read_str = re.sub('\^.', '', read_str) #remove caret and next symbol
        read_str = re.sub('\$', '', read_str ) #remove dollar
        
        read_str = re.sub('\,', '.', read_str) #substitute comma with dot
        read_str = read_str.upper() #switch all letters to uppercase
        
        return read_str
        
        
    def prepare_read_vars(self, read_str):    
        skip_index = 0
        
        for i in range(0, len(read_str)):
            if(i<skip_index) : continue #skip indel symbols
             
            if(read_str[i] == '+' or read_str[i] == '-'):
                self.ins_cnt += 1
                num_len = 0
                for j in range (i+1, len(read_str)):
                    if(read_str[j].isnumeric()): num_len +=1
                    else: break
                indel_len = int(read_str[i+1 : i+1+num_len])
#                 if(read_str[i-1] == '.'): 
                new_indel = read_str[(i):(i+num_len+indel_len+1)]
#                 else:
#                     new_ins = read_str[i-1] + read_str[(i+2):(i+2+ins_len)]
                #cur_cnt = self.ins_seqs.get(new_ins, 0)
                #self.ins_seqs[new_ins] = cur_cnt + 1
                if new_indel in self.read_vars : self.read_vars.update({new_indel:self.read_vars[new_indel]+1})
                else : self.read_vars.update({new_indel:1})
                
                skip_index = i + num_len + indel_len +1
            
            if(read_str[i] == '.'): self.read_vars.update({'.' :self.read_vars['.']+1})
            if(read_str[i] == 'A'): self.read_vars.update({'A' :self.read_vars['A']+1})
            if(read_str[i] == 'T'): self.read_vars.update({'T' :self.read_vars['T']+1})
            if(read_str[i] == 'C'): self.read_vars.update({'C' :self.read_vars['C']+1})
            if(read_str[i] == 'G'): self.read_vars.update({'G' :self.read_vars['G']+1})
        
        self.read_vars = OrderedDict(sorted(self.read_vars.items(), key=lambda item: item[1], reverse=True))
        self.read_vars_list = list(self.read_vars.items())
        
        print('prepare_read_vars')
#         print(self.read_vars)
        print(self.read_vars_list)
    
    def process_read_line(self, read_str):
        self.find_variants(read_str)
        #read_str = self.remove_start_stop_signs(read_str)
        
        return read_str
    
    
    
  

SyntaxError: invalid syntax (<ipython-input-205-b6e0602c8273>, line 77)

In [178]:
from collections import OrderedDict
d = { '123': 1,
      '124': 15,
      '125': 7,
    }
# d_ascending = OrderedDict(sorted(d.items(), key=lambda kv: kv[1]['key3']))
d_descending = OrderedDict(sorted(d.items(), key=lambda item: item[1], reverse=True))

print(d_descending)

OrderedDict([('124', 15), ('125', 7), ('123', 1)])


In [203]:
##
## MAIN CODE - opens files, goes through the pileup file, picks up 
##

with open (pileup_file_str, 'r') as pileup_file, open (output_file_temp_str, 'w') as temp, open (reference_fai_file_str, 'r') as fai:
    
    temp = write_vcf_header(temp, fai)
    
    line_num = 0;
    for line in iter(pileup_file.readline, ''):
        line_num+=1 
        #if (line_num > 3): break
        base_pileup = PileupLine(line)
        res_str = str(base_pileup.res_str)
        processed_str = base_pileup.process_read_line(res_str)
        #if(line_num == 1): #DEBUG 
            #if(base_pileup.cnt == 0):
                #
        temp.write(processed_str + '\n')                 
                
        
        #TODO
        # get variants from pileup line
        # print them out
        # go to the next one
                
                
                
                

prepare_read_vars
OrderedDict([('A', 6), ('+2AA', 1), ('.', 0), ('T', 0), ('C', 0), ('G', 0)])
[('A', 6), ('+2AA', 1), ('.', 0), ('T', 0), ('C', 0), ('G', 0)]
prepare_read_vars
OrderedDict([('.', 5), ('+1G', 3), ('A', 1), ('G', 1), ('+1A', 1), ('-1G', 1), ('-1T', 1), ('-1C', 1), ('T', 0), ('C', 0)])
[('.', 5), ('+1G', 3), ('A', 1), ('G', 1), ('+1A', 1), ('-1G', 1), ('-1T', 1), ('-1C', 1), ('T', 0), ('C', 0)]
prepare_read_vars
OrderedDict([('.', 49), ('A', 1), ('+2AG', 1), ('-3GTA', 1), ('T', 0), ('C', 0), ('G', 0)])
[('.', 49), ('A', 1), ('+2AG', 1), ('-3GTA', 1), ('T', 0), ('C', 0), ('G', 0)]
prepare_read_vars
OrderedDict([('.', 49), ('A', 1), ('+2AG', 1), ('-3GTA', 1), ('T', 0), ('C', 0), ('G', 0)])
[('.', 49), ('A', 1), ('+2AG', 1), ('-3GTA', 1), ('T', 0), ('C', 0), ('G', 0)]
find_variants
var
var
REF
SNV
find_variants
var
REF
find_variants
var
REF
find_variants
var
REF
find_variants
var
REF
find_variants
var
REF
find_variants
var
REF
find_variants
var
REF
find_variants
var
REF
find

In [194]:
##
## TESTS - test different pileup lines 
##


In [85]:
collections.Counter(".$.$...,..........,...........,.,,,,,..,......,,,....,,,.,").most_common() 

[('.', 40), (',', 16), ('$', 2)]